# Collaborative-Filtering

### CREATED BY : **ANDREAS MULYARHARDJA**
<br><br>
In this section, we use Collaborative-Filteringg to recomend movies to users in our data. There are 2 approach used here, by using manual (pearson correlation) and by using <i>surprise</i> modules in Python <i>K-Nearest Neighbour with Means</i>. We use <i><b>RatingUser.csv</b></i> for our data. We try to give recomendation movies to user <b>Mulya</b> using this method.<br><br>

## Import Modules
<br>

In [77]:
import pandas as pd
import numpy as np

# For manual Colaborative Filtering
from math import sqrt
from scipy import spatial

# For Colaborative Filtering using Modules

from surprise import Dataset
from surprise import Reader

from surprise import KNNWithMeans

## Data Preprocessing

<br><br><br>
First we read data and preprocess it. For this <i><b>repositories</b></i> we do not deep down to EDA as building recomendation algorithm is our main focus here.

In [97]:
dataset=pd.read_csv('RatingUser.csv',delimiter=",")
df=dataset
dataset=dataset.set_index('Nama Anda')
del dataset.index.name
dataset=dataset.fillna(-1) # will be discussed in the next markdown
dataset.head()

,Wall-e,The Terminator,The Wizard of Oz,Jaws,Gladiator,The Avengers,Zootopia,Aladdin,Spider-Man: Homecoming,Guardians of the Galaxy Vol. 2
Hania,3.0,5.0,4.0,4.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0
Topik Zulkarnain,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,5.0,-1.0,4.0,2.0
AhokTemanFirli,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,4.0
franadek,4.0,4.0,4.0,5.0,3.0,5.0,4.0,5.0,4.0,4.0
OM INDRA,3.0,-1.0,2.0,-1.0,5.0,5.0,-1.0,1.0,5.0,5.0


<br><br>
For our manual collaborative to be run succesfully, we need to convert our dataset into dictionaries with Key is name of users, and Key values is dictionary of Movies and its ratings. We also have to remove movies that was not been watched by users. To make things easier, we change NaN values to -1.

In [98]:
dataset = dataset.to_dict(orient='index')
for name in dataset:
    dicto={}
    for movie in dataset[name]:
        if dataset[name][movie]!=-1:
            dicto[movie]=dataset[name][movie]
    dataset[name]=dicto
dataset
        


{'Hania': {'Wall-e': 3.0,
  'The Terminator': 5.0,
  'The Wizard of Oz': 4.0,
  'Jaws': 4.0,
  'Gladiator': 4.0},
 'Topik Zulkarnain': {'The Avengers': 5.0,
  'Zootopia': 5.0,
  'Spider-Man: Homecoming': 4.0,
  'Guardians of the Galaxy Vol. 2': 2.0},
 'AhokTemanFirli': {'The Avengers': 3.0,
  'Guardians of the Galaxy Vol. 2': 4.0},
 'franadek': {'Wall-e': 4.0,
  'The Terminator': 4.0,
  'The Wizard of Oz': 4.0,
  'Jaws': 5.0,
  'Gladiator': 3.0,
  'The Avengers': 5.0,
  'Zootopia': 4.0,
  'Aladdin': 5.0,
  'Spider-Man: Homecoming': 4.0,
  'Guardians of the Galaxy Vol. 2': 4.0},
 'OM INDRA': {'Wall-e': 3.0,
  'The Wizard of Oz': 2.0,
  'Gladiator': 5.0,
  'The Avengers': 5.0,
  'Aladdin': 1.0,
  'Spider-Man: Homecoming': 5.0,
  'Guardians of the Galaxy Vol. 2': 5.0},
 'Mulya': {'The Avengers': 4.0,
  'Zootopia': 2.0,
  'Aladdin': 5.0,
  'Spider-Man: Homecoming': 4.0,
  'Guardians of the Galaxy Vol. 2': 3.0},
 'Genjeh': {'Wall-e': 5.0,
  'The Terminator': 4.0,
  'The Wizard of Oz': 3.0,


<br><br>
Now our data is ready for Colaborative Filtering.<br><br><br>

## Manual Colaborative Filtering using Pearson Correlation

<br>
Next, we defining function to calculate distance and simmilarity between users, and to do collaborative clustering. Here, we use <b>Pearson Correlation</b> for determining similarity between users that is :

![asd](https://wikimedia.org/api/rest_v1/media/math/render/svg/869d208d19ba2481a306aa5b2829d6f147215f22)

After we find which users is highly correlated to which, we can predict ratings of movies we have not watched from those similar users that already have watched. <br><br>

In [99]:
#import code 

def person_correlation(person1, person2):

   # To get both rated items
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    # Checking for ratings in common
    if number_of_ratings == 0:
        return 0

    # Add up all the preferences of each user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Calculate the pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

def most_similar_users(person, number_of_users):

    # returns the number_of_users (similar persons) for a given specific person
    scores = [(person_correlation(person, other_person), other_person) for other_person in dataset if other_person != person]

    # Sort the similar persons so the highest scores person will appear at the first
    scores.sort()
    scores.reverse()
    return scores[0:number_of_users]
        
def user_recommendations(person):

    # Gets recommendations for a person by using a weighted average of every other user's rankings
    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        # don't compare me to myself
        if other == person:
            continue
        sim = person_correlation(person,other)
        #print ">>>>>>>",sim

        # ignore scores of zero or lower
        if sim <=0: 
            continue
        for item in dataset[other]:

            # only score movies i haven't seen yet
            if item not in dataset[person] or dataset[person][item] == 0:

                # Similrity * score
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                # sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+= sim

        # Create the normalized list

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    # returns the recommended items
    recommendataions_list = [recommend_item for score,recommend_item in rankings]
    return recommendataions_list,rankings
        



<br><br>
Now we call user_recomendation function and tried to give movie recomendation to user <b>Mulya</b>.  

In [124]:
name='Mulya'
title,rating=user_recommendations(name)
print('\nRecomendation movie and its predicted ratings for user {} is \n {}'.format(name,str(rating)))


Recomendation movie and its predicted ratings for user Mulya is 
 [(4.3039040240550515, 'Jaws'), (3.811939905577606, 'The Terminator'), (3.780128839373005, 'Wall-e'), (3.541676960203368, 'Gladiator'), (3.4690682979474237, 'The Wizard of Oz')]


<br><br>
From here, we see that all the 5 films has predicted good ratings. If we take only those movies who has predicted ratings more than 3.5, we have Jaws, The Terminator, Wall-e, and Gladiator. <br><br><br>

We now want to see the simmilarities within each user with <b>Mulya</b>. We see that 'luck' has Pearson Correlation 1.0, which is highly correlated. <br>

In [133]:
most_similar_users('Mulya',24)

[(1.0, 'luck'),
 (0.8807048459279792, 'Jawaharal'),
 (0.8660254037844402, 'Putrisqiana'),
 (0.8164965809277261, 'Febi ganteng gak ada obat'),
 (0.7844645405527345, 'Rima'),
 (0.7205766921228899, 'franadek'),
 (0.7071067811865475, 'Genjeh'),
 (0, 'jul'),
 (0, 'faizah'),
 (0, 'bunga'),
 (0.0, 'Topik Zulkarnain'),
 (0, 'Star'),
 (0, 'Romantika'),
 (0, 'Indra Junior'),
 (0, 'Indra 1991 SM'),
 (0, 'Hania'),
 (0, 'Dpv'),
 (0, 'Damar Teman Firli'),
 (0.0, 'ANI'),
 (-0.1961161351381871, 'Nonton_Saat_Diskon'),
 (-0.8164965809277261, 'OM INDRA'),
 (-1.0, 'Maria O.'),
 (-1.0, 'AhokTemanFirli')]

<br><br>
If we inspect 'top 2 similar users, we get :

In [126]:
df[df['Nama Anda']=='Mulya'].dropna(axis=1) 

,Nama Anda,The Avengers,Zootopia,Aladdin,Spider-Man: Homecoming,Guardians of the Galaxy Vol. 2
5,Mulya,4.0,2.0,5.0,4.0,3.0


In [132]:
df[df['Nama Anda']=='luck'].dropna(axis=1) 

,Nama Anda,Wall-e,The Avengers,Guardians of the Galaxy Vol. 2
22,luck,3.0,4.0,2.0


In [131]:
df[df['Nama Anda']=='Jawaharal'].dropna(axis=1) 

,Nama Anda,Wall-e,The Terminator,The Wizard of Oz,Gladiator,The Avengers,Zootopia,Aladdin,Spider-Man: Homecoming,Guardians of the Galaxy Vol. 2
15,Jawaharal,2.0,4.0,3.0,5.0,5.0,4.0,5.0,5.0,4.0


<br><br>
Because Pearson correlation is measuring how much the users is correlated <b><i>LINEARLY</i></b>, using pearson user : luck and user : Mulya is measured to be on the same ' line '. Colaborative Filtering is calculated predicted ratings base on similar user which translates to '<i>Has the same taste</i>' to already watched movies. Other than pearson, we can also use Cosine to calculate similarity. <br><br><br><br><br>

## Using KNNWithMeans Modules surprise

<br><br>
Now, we use Python modules to do colaborative clustering. Compared to above, we use Pearson and Cosine with KNNBasic that is basicaly Colaborative Filtering used above. In Surprise, we can use alot of method like KNNWithMeans, NormalPredictor, and etc. Compared to pearson, Cosine calculated its similarities by :

![](https://miro.medium.com/max/306/1*gjQMBuTqNENrj4Md_Y8Rzg.png)

<br>
We can intepret Cosine as the angle between vectors. So if 2 users, e.g. a,b have close angle, its cosine will equal to 1 (because its angle is 0 and cos(0)=1). We compared each method below

### Preprocessing
<br><br>
Same as above section, we preprocess our data which required to be ' stacked '. By default, NaN values will be dropped so we do not need to fill NaN values

In [197]:
data=pd.read_csv('RatingUser.csv',delimiter=',')
df = (data.set_index(['Nama Anda']).stack().reset_index(name='a'))
df.columns=['Nama Anda','Movie','Rating']
df[df['Nama Anda']=='Mulya']

,Nama Anda,Movie,Rating
28,Mulya,The Avengers,4.0
29,Mulya,Zootopia,2.0
30,Mulya,Aladdin,5.0
31,Mulya,Spider-Man: Homecoming,4.0
32,Mulya,Guardians of the Galaxy Vol. 2,3.0


<br><br> And That's it! now we deep dive into Colaborative-Filtering.

### Colaborative FIltering

In [199]:
reader = Reader(rating_scale=(1, 5))

# Loads Pandas dataframe
data = Dataset.load_from_df(df, reader)

In [200]:
from surprise import KNNWithMeans
from surprise import KNNBasic
# To use item-based cosine similarity
sim_options1 = {
    "name": "pearson",
    "user_based": True,  # Compute  similarities between items
}
sim_options2 = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between items
}
algo = KNNBasic(sim_options=sim_options1)
algo2 = KNNBasic(sim_options=sim_options2)
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)
algo2.fit(trainingSet)


Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [201]:
Name='Mulya'

Movie=df['Movie'].unique()
rec=[]
for i in Movie:
    movie_watched = np.array(df[df['Nama Anda']==Name]['Movie'])
    if i not in movie_watched:
        prediction = algo.predict(Name, i)
        rec.append([i,Name,float(prediction.est)])
rec=pd.DataFrame(rec)
rec.columns=['Movie','User','Predicted Rating']

print(rating) # Using Manual
rec.sort_values('Predicted Rating',ascending=False) # Using KNNBasic


[(4.3039040240550515, 'Jaws'), (3.811939905577606, 'The Terminator'), (3.780128839373005, 'Wall-e'), (3.541676960203368, 'Gladiator'), (3.4690682979474237, 'The Wizard of Oz')]


,Movie,User,Predicted Rating
3,Jaws,Mulya,4.303904
1,The Terminator,Mulya,3.811940
0,Wall-e,Mulya,3.780129
4,Gladiator,Mulya,3.541677
2,The Wizard of Oz,Mulya,3.469068


<br><br>
We see that using KNNBasic, we get same result as using manual method above. Now using cosine to calculate simmilar result, we get :

In [203]:
Name='Mulya'

Movie=df['Movie'].unique()
rec=[]
for i in Movie:
    movie_watched = np.array(df[df['Nama Anda']==Name]['Movie'])
    if i not in movie_watched:
        prediction = algo2.predict(Name, i)
        rec.append([i,Name,float(prediction.est)])
rec=pd.DataFrame(rec)
rec.columns=['Movie','User','Predicted Rating']
rec.sort_values('Predicted Rating',ascending=False)




,Movie,User,Predicted Rating
3,Jaws,Mulya,4.599518
0,Wall-e,Mulya,3.942756
4,Gladiator,Mulya,3.729781
1,The Terminator,Mulya,3.727418
2,The Wizard of Oz,Mulya,3.601987


<br><br>
These are different prediction compared to above. This is because similarities matrix using pearson and cosine is completely difference.

In [207]:
a=algo.compute_similarities()
Sim1=pd.DataFrame(a,index=df['Nama Anda'].unique().tolist(),columns=df['Nama Anda'].unique().tolist())['Mulya']

b=algo2.compute_similarities()
Sim2=pd.DataFrame(b,index=df['Nama Anda'].unique().tolist(),columns=df['Nama Anda'].unique().tolist())['Mulya']

Sim3 = pd.DataFrame([Sim1,Sim2],index=['Pearson','Cosine']).transpose()
Sim3

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


,Pearson,Cosine
Hania,0.000000,0.000000
Topik Zulkarnain,0.000000,0.926506
AhokTemanFirli,-1.000000,0.960000
franadek,0.720577,0.977964
OM INDRA,-0.816497,0.847174
Mulya,1.000000,1.000000
Genjeh,0.707107,0.992302
faizah,0.000000,1.000000
Dpv,0.000000,0.991837
Putrisqiana,0.866025,0.993884


<br><br>
For example, we see that user Mulya and Topik Zulkarnain has different correlation result, using pearson and cosine. But there are also few user that is quite similar like user luck. If we see our original data of these 2 account and compared it to Mulya we get :

In [210]:
compar=pd.DataFrame()
print(df[df['Nama Anda']=='Mulya'])
print(df[df['Nama Anda']=='Topik Zulkarnain'])
print(df[df['Nama Anda']=='luck'])

   Nama Anda                           Movie  Rating
28     Mulya                    The Avengers     4.0
29     Mulya                        Zootopia     2.0
30     Mulya                         Aladdin     5.0
31     Mulya          Spider-Man: Homecoming     4.0
32     Mulya  Guardians of the Galaxy Vol. 2     3.0
          Nama Anda                           Movie  Rating
5  Topik Zulkarnain                    The Avengers     5.0
6  Topik Zulkarnain                        Zootopia     5.0
7  Topik Zulkarnain          Spider-Man: Homecoming     4.0
8  Topik Zulkarnain  Guardians of the Galaxy Vol. 2     2.0
    Nama Anda                           Movie  Rating
132      luck                          Wall-e     3.0
133      luck                    The Avengers     4.0
134      luck  Guardians of the Galaxy Vol. 2     2.0


<br><br>
You see that using intuitive, user Mulya is similiar with Topik Zukarnain and kinda difference between Luck. It looks like Cosine is better at calculated similarities compared to pearson. So, we use collaborative filtering using cosine.
<br><br>
If, for instances we only recomended movie that has predicted rating more than 3.9, we can get recomendation to user Mulya 
<br><br>

In [214]:
rec_user=rec[rec['Predicted Rating']>=3.9]
print('User {} is recomended to watch {}'.format(Name,rec_user['Movie'].tolist()))

User Mulya is recomended to watch ['Wall-e', 'Jaws']


## Conclusion
<br><br>
<br>
We can conclude that Using colabrive Filtering, we get recomendation movies for user Mulya that is <b>Wall-e</b> and <b>Jaws</b>. Using Pearson vs Cosine, in general we see that cosine is better because pearson using corelated based on ' how strong linear relationship between users ' while cosine is using ' angle ' between users. We also see that in some data, Cosine is more sense than Pearson.